# SQL Database Schema Report

## Parameters
* SQLInstance
* DBName

## Data sets

### Parent Table


In [0]:
SELECT [t].Object_id
     , SCHEMA_NAME([t].schema_id) AS schema_name
     , [t].[name] AS [table_name]
     , [t].[create_date] AS [created]
     , [t].[modify_date] AS [last_modified]
     , [p].[rows] AS [num_rows]
     , [ep].value AS [comments]
FROM [sys].[tables] AS [t]
     LEFT JOIN [sys].[extended_properties] AS [ep]
     ON [t].object_id = [ep].[major_id]
        AND [ep].[name] = 'MS_Description'
        AND [ep].[minor_id] = 0
        AND [ep].[class_desc] = 'OBJECT_OR_COLUMN'
outer apply
     (
         SELECT [pt].object_id
              , SUM([pt].[rows]) AS [rows]
         FROM [sys].[partitions] AS [pt]
              where [pt].[object_id] = [t].[object_id]
			  and [pt].index_id = (SELECT MIN(index_id) FROM [sys].[partitions] WHERE [object_id] = [t].OBJECT_ID)
         GROUP BY [pt].object_id
     ) AS [p]
WHERE OBJECTPROPERTY([t].OBJECT_ID, 'IsUserTable') = 1 

M Script  to clean up result set

In [0]:
let
    Source = Sql.Database(SQLInstance, DBName, [Query="SELECT [t].Object_id#(lf)     , SCHEMA_NAME([t].schema_id) AS schema_name#(lf)     , [t].[name] AS [table_name]#(lf)     , [t].[create_date] AS [created]#(lf)     , [t].[modify_date] AS [last_modified]#(lf)     , [p].[rows] AS [num_rows]#(lf)     , [ep].value AS [comments]#(lf)FROM [sys].[tables] AS [t]#(lf)     LEFT JOIN [sys].[extended_properties] AS [ep]#(lf)     ON [t].object_id = [ep].[major_id]#(lf)        AND [ep].[name] = 'MS_Description'#(lf)        AND [ep].[minor_id] = 0#(lf)        AND [ep].[class_desc] = 'OBJECT_OR_COLUMN'#(lf)outer apply#(lf)     (#(lf)         SELECT [pt].object_id#(lf)              , SUM([pt].[rows]) AS [rows]#(lf)         FROM [sys].[partitions] AS [pt]#(lf)              where [pt].[object_id] = [t].[object_id]#(lf)#(tab)#(tab)#(tab)  and [pt].index_id = (SELECT MIN(index_id) FROM [sys].[partitions] WHERE [object_id] = [t].OBJECT_ID)#(lf)         GROUP BY [pt].object_id#(lf)     ) AS [p]#(lf)WHERE OBJECTPROPERTY([t].OBJECT_ID, 'IsUserTable') = 1 ", CreateNavigationProperties=false]),
    #"Duplicated Column" = Table.DuplicateColumn(Source, "table_name", "table_name - Copy"),
    #"Split Column by Delimiter" = Table.SplitColumn(#"Duplicated Column", "table_name - Copy", Splitter.SplitTextByEachDelimiter({"_"}, QuoteStyle.Csv, true), {"table_name - Copy.1", "table_name - Copy.2"}),
    #"Removed Columns" = Table.RemoveColumns(#"Split Column by Delimiter",{"table_name - Copy.1"}),
    #"Renamed Columns" = Table.RenameColumns(#"Removed Columns",{{"table_name - Copy.2", "Table Suffix"}, {"num_rows", "Parent Rows"}, {"table_name", "Parent Table"}, {"schema_name", "Parent Schema"}}),
    #"Sorted Rows" = Table.Sort(#"Renamed Columns",{{"Parent Table", Order.Ascending}}),
    #"Added Custom" = Table.AddColumn(#"Sorted Rows", "Parent Table FQ", each "[" & [Parent Schema] & "].[" & [Parent Table] & "]")
in
    #"Added Custom"

### Child Table

In [0]:
SELECT [t].Object_id
     , SCHEMA_NAME([t].schema_id) AS schema_name
     , [t].[name] AS [table_name]
     , [p].[rows] AS [num_rows]
FROM [sys].[tables] AS [t]
outer apply
     (
         SELECT [pt].object_id
              , SUM([pt].[rows]) AS [rows]
         FROM [sys].[partitions] AS [pt]
              where [pt].[object_id] = [t].[object_id]
			  and [pt].index_id = (SELECT MIN(index_id) FROM [sys].[partitions] WHERE [object_id] = [t].OBJECT_ID)
         GROUP BY [pt].object_id
     ) AS [p]
WHERE OBJECTPROPERTY([t].OBJECT_ID, 'IsUserTable') = 1 

M script to do cleanup

In [0]:
let
    Source = Sql.Database(SQLInstance, DBName, [Query="SELECT [t].Object_id#(lf)     , SCHEMA_NAME([t].schema_id) AS schema_name#(lf)     , [t].[name] AS [table_name]#(lf)     , [p].[rows] AS [num_rows]#(lf)FROM [sys].[tables] AS [t]#(lf)outer apply#(lf)     (#(lf)         SELECT [pt].object_id#(lf)              , SUM([pt].[rows]) AS [rows]#(lf)         FROM [sys].[partitions] AS [pt]#(lf)              where [pt].[object_id] = [t].[object_id]#(lf)#(tab)#(tab)#(tab)  and [pt].index_id = (SELECT MIN(index_id) FROM [sys].[partitions] WHERE [object_id] = [t].OBJECT_ID)#(lf)         GROUP BY [pt].object_id#(lf)     ) AS [p]#(lf)WHERE OBJECTPROPERTY([t].OBJECT_ID, 'IsUserTable') = 1 ", CreateNavigationProperties=false]),
    #"Renamed Columns" = Table.RenameColumns(Source,{{"schema_name", "Child Schema"}, {"table_name", "Child Table"}, {"num_rows", "Child Rows"}}),
    #"Added Custom" = Table.AddColumn(#"Renamed Columns", "Child Table FQ", each "[" & [Child Schema] & "].[" & [Child Table] & "]")
in
    #"Added Custom"

More detailed table data set

In [0]:
SELECT @@SERVERNAME AS [Server Name]
     , DB_NAME() AS [Database Name]
     , SCHEMA_NAME(schema_id) AS [Schema Name]
     , [T].[name] AS [Table Name]
     , SUM([Partitions].[rows]) AS [RowCount]
	 , MAX(I.[Table Type]) as [Table Type]
	 , MAX(I.[Index Count]) as [IndexCount]
     , CAST(CONVERT(VARCHAR, current_timestamp, 112) AS INT) AS [SnapshotDateKey]
     , CAST(replace(LEFT(CONVERT(VARCHAR, current_timestamp, 14), 8), ':', '') AS INT) AS [SnapshotTimeKey]
FROM [sys].[tables] AS [T]
     JOIN [sys].[partitions] AS [Partitions]
     ON [T].[object_id] = [Partitions].[object_id]
        AND [Partitions].[index_id] IN(0, 1)
Left outer join (select object_id
	, MAX(case [type] when 0 then 'Heap'
			when 1 then 'Clustered' else null end) as [Table Type] 
	, sum(case when [type] <> 0 then 1 else 0 end) as [Index Count]
from sys.indexes
group by object_id ) as I
on T.[object_id] = I.[object_id]
WHERE [T].[is_ms_shipped] = 0
GROUP BY SCHEMA_NAME(schema_id)
       , [T].[name];

## DB Columns


In [0]:
SELECT OBJECT_NAME([sc].Object_id) AS [Table Name]
     , [sc].Object_id
     , [sc].[column_id] AS [Seq]
     , [sc].[name] AS [Column Name]
	 , CASE WHEN [sc].[column_id] = 1 THEN '' ELSE ',' END +'[' + [sc].[name] +'] ' 
		+   TYPE_NAME([sc].[system_type_id]) 
		+ CASE WHEN _.[format_len] = 0 THEN ''
					ELSE '(' + case when _.[Length] = -1 then 'MAX'
						when _.[Prec] = '     ' 
						then cast(case when TYPE_NAME([sc].[system_type_id]) in (N'NCHAR', N'NVARCHAR') then _.[Length]/2 else _.[Length] end as varchar)
						else rtrim( _.[Prec] ) + ', ' + rtrim(_.[Scale])
						end
					 + ')' END
	+ CASE [sc].[is_nullable] WHEN 0
		THEN ' NOT NULL' 
		ELSE ' NULL' END AS [ColDef]
     , TYPE_NAME([sc].[user_type_id]) AS [User Type Name]
     , TYPE_NAME([sc].[system_type_id]) AS [Type Name]
     , CASE
           WHEN TYPE_NAME([sc].[system_type_id]) IN (N'NCHAR', N'NVARCHAR') AND [_].[Length] <> -1
           THEN [_].[Length] / 2
           ELSE [_].[Length]
       END AS [Length]
     , [_].[Prec]
     , [_].[Scale]
     , TYPE_NAME([sc].[system_type_id])
       + CASE
             WHEN [_].[format_len] = 0
             THEN ''
             ELSE '('
                  + CASE WHEN [_].[Length] = -1 THEN 'MAX'
                        WHEN [_].[Prec] = '     '
                        THEN CAST(CASE
                                      WHEN TYPE_NAME([sc].[system_type_id]) IN(N'NCHAR', N'NVARCHAR')
                                      THEN [_].[Length] / 2
                                      ELSE [_].[Length]
                                  END AS VARCHAR)
                        ELSE RTRIM([_].[Prec]) + ', ' + RTRIM([_].[Scale])
                    END
                  + ')'
         END AS [Data Type]
     , [sc].[is_nullable] AS [Is Nullable]
     , [sc].[is_identity] AS [Is Identity]
     , [sc].[is_computed] AS [Is Computed]
     , [sc].[is_ansi_padded] AS [Is ANSI Padded]
     , ISNULL(OBJECT_NAME([sc].[default_object_id]), '') AS [Default Name]
     , ISNULL(
       (
           SELECT TOP 1 SUBSTRING(text, 2, LEN([text]) - 2)
           FROM [syscomments](NOLOCK)
           WHERE [id] = [sc].[default_object_id]
       ), '') AS [DefaultValue]
     , [ep].value AS [comments]
FROM [sys].[all_columns](NOLOCK) AS [SC]
     LEFT JOIN [sys].[extended_properties] AS [ep]
     ON [sc].object_id = [ep].[major_id]
        AND [ep].[name] = 'MS_Description'
        AND [ep].[minor_id] = [sc].[column_id]
        AND [ep].[class_desc] = 'OBJECT_OR_COLUMN'
     OUTER APPLY
     (
         SELECT CONVERT(INT, [sc].[max_length]) AS [Length]
              , CASE
                    WHEN CHARINDEX(TYPE_NAME([sc].[system_type_id]) + ',', N'tinyint,smallint,decimal,int,bigint,real,money,float,numeric,smallmoney,date,time,datetime2,datetimeoffset,') > 0
                    THEN CONVERT(CHAR(5), COLUMNPROPERTY(object_id, [sc].[name], 'precision'))
                    ELSE '     '
                END AS [Prec]
              , CASE
                    WHEN CHARINDEX(TYPE_NAME([system_type_id]) + ',', N'tinyint,smallint,decimal,int,bigint,real,money,numeric,smallmoney,date,time,datetime2,datetimeoffset,') > 0
                    THEN CONVERT(CHAR(5), OdbcScale( [sc].[system_type_id], [sc].[scale]) )
                    ELSE '     '
                END AS [Scale]
              , CASE
                    WHEN CHARINDEX(TYPE_NAME([sc].[system_type_id]), N'decimal,real,numeric,datetimeoffset,char,varchar,nchar,nvarchar,varbinary') > 0
                    THEN 1
                    ELSE 0
                END AS [format_len]
     ) AS [_]
WHERE OBJECTPROPERTY([sc].OBJECT_ID, 'IsUserTable') = 1

### Relationships



In [0]:
SELECT [CC].[parent_object_id] AS [Parent_object_id]
		 , [CC].[referenced_object_id]
		 , OBJECT_NAME([cc].constraint_object_id) AS ConstraintName
		 , STUFF(
		   (
			   SELECT ', ' + QUOTENAME(OBJECT_NAME([C].[object_id])) + '.'+  QUOTENAME([C].[name]) + ' = ' + QUOTENAME(OBJECT_NAME([A].[referenced_object_id]))+ '.' +QUOTENAME([P].[name])
			   FROM [sys].[foreign_key_columns] AS [A]
					JOIN [sys].[columns] AS [C]
					ON [C].[object_id] = [A].[parent_object_id]
					   AND [C].[column_id] = [A].[parent_column_id]
					JOIN [sys].[columns] AS [P]
					ON [P].[object_id] = [A].[referenced_object_id]
					   AND [P].[column_id] = [A].[referenced_column_id]
			   WHERE [CC].[parent_object_id] = [A].[parent_object_id]
					 AND [CC].[referenced_object_id] = [A].[referenced_object_id]
					 AND [CC].constraint_object_id = [A].constraint_object_id
			   FOR XML PATH('')
		   ), 1, 2, '') AS [ConstraintColumns]
	FROM [sys].[foreign_key_columns] AS [CC]
	GROUP BY [CC].[parent_object_id]
		   , [CC].[referenced_object_id]
		   ,[cc].constraint_object_id

### Model relationships

> DB Columns(Object_id) <- Parent Table(Object_id)

> Relationships(Parent_Object_id) <- Parent Table(Object_id)

> Relationships(References_object_id) <- Child Table(Object_id)